## Objectif du notebook :

Créer des fonctions qui oeuvreront au même moment que la NER pour protéger des instances sensibles qui ne sont pas des entités nommées, comme !
- Des numéros de sécurité sociale
- Des sites internet
- Des adresses mail
- Des plaques d'immatriculation

In [320]:
# Imports

import regex as re

In [321]:
# Création de la liste de groupes d'entités :

entity_groups = ["SS", "email", "plate", "phone", "date"]

In [322]:
# Regex pour les NIR :

reg_ss = r"/^[12][0-9]{2}[0-1][0-9](2[AB]|[0-9]{2})[0-9]{2}[0-9]{3}[0-9]{3}$/x"

# Regex pour les plaques :

reg_plate = r"[A-Z]+[A-Z0-9\- ]{6,8}"
#reg_plate = r"\b[0-9A-Z]{3}([^ 0-9A-Z]|\s)?[0-9]{4}\b"

# Regex pour les numéros de téléphone :

reg_phone = r"(0|(\+33)|(0033))+[1-9]+[0-9]{8}"

# Regex pour les numéros

reg_num = r"[0-9+/.\- ]{6,}"

# Regex pour un email :

reg_email = r"[a-zA-Z0-9.\-\_]+@[a-zA-Z0-9.\-\_]+"

# Regex pour les dates :

reg_dates = r"^(?:(?:31(\/|-|\.)(?:0?[13578]|1[02]))\1|(?:(?:29|30)(\/|-|\.)(?:0?[13-9]|1[0-2])\2))(?:(?:1[6-9]|[2-9]\d)?\d{2})$|^(?:29(\/|-|\.)0?2\3(?:(?:(?:1[6-9]|[2-9]\d)?(?:0[48]|[2468][048]|[13579][26])|(?:(?:16|[2468][048]|[3579][26])00))))$|^(?:0?[1-9]|1\d|2[0-8])(\/|-|\.)(?:(?:0?[1-9])|(?:1[0-2]))\4(?:(?:1[6-9]|[2-9]\d)?\d{2})"

In [323]:
# faire fonction générale qui prend un dico en entrée et qui run toutes les fonctions et ajoute tout au bon format au dictionnaire et un texte

def add_entities (text:str,
                  l : list = None):
    if not l:
        l = []
    plates = re.findall(reg_plate, text)
    plates = list(dict.fromkeys(plates))
    numbers = re.findall(reg_num, text)
    numbers = list(dict.fromkeys(numbers))
    emails = re.findall(reg_email, text)
    emails = list(dict.fromkeys(emails))
    entity_groups = {"Number":numbers,"email":emails,"plate":plates}
    
    for entity_type in entity_groups.keys():
        for item in entity_groups[entity_type]:
            if isinstance(item, tuple) and tuple[0] != '':
                dic = {}
                dic["entity_group"] = entity_type
                dic["word"] = item[0]
                l.append(dic)
            elif isinstance(item, str):
                dic = {}
                dic["entity_group"] = entity_type
                dic["word"] = item
                l.append(dic)
    return(l)

In [324]:
txt = "OL-077-DV 2 69 24 18 158 275 lucie@clemot.com +33622541979 24-06-2000"

add_entities(txt)


[{'entity_group': 'Number', 'word': ' 2 69 24 18 158 275 '},
 {'entity_group': 'Number', 'word': ' +33622541979 24-06-2000'},
 {'entity_group': 'email', 'word': 'lucie@clemot.com'},
 {'entity_group': 'plate', 'word': 'OL-077-DV '}]

In [325]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

from transformers import pipeline

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

def identify_ners(text:str) -> list :
    ner = nlp(text)
    ner = add_entities(text, ner)
    return(ner)
    

def pseudo(name:str):
    return('X'*len(name))

def pseudo_all(text : str , file_ners: list):
    """
    Pseudonymizes all NERs within the file."""
    for i, ner in enumerate(file_ners):
        if ner["entity_group"]:
            name = ner["word"]
            if len(name) > 1:
                pseudo_name = pseudo(name)
                text = text.replace(name, pseudo_name)
    return(text)

In [326]:
text = "j'habite au 2 route tru truc à machin"

ners = identify_ners(text)

text_new = pseudo_all(text, ners)

text_new

"j'habite au 2 route XXXXXXXX à XXXXXX"

In [327]:
import os

import pandas as pd
import s3fs
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "projet-transformers"
FILE_KEY_S3 = "01092023 - Tests Anonymisation - Outil DINUM 2.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

In [336]:
with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df_dila = pd.read_csv(file_in, 
            sep=";",
            encoding='utf-8',
            usecols = ["N°","Message d'origine"],
            index_col=0
            )

df_dila.head()

,Message d'origine
N°,
1,""" Bonjour, je viens d'arriver en france ,j'aie..."
2,""" J'ai fait une demande de duplicata de carte ..."
3,""" Bonjour, J'ai passé commande pour certificat..."
4,""" Bonjour, Mr LE BRUYN Steeve est inscrit chez..."
5,""" En date du 4/01/218, j'ai commandé deux vign..."


In [337]:
def process_text(sentence :str):
    ners = identify_ners(sentence)
    new_sentence = pseudo_all(sentence, ners)
    return(new_sentence)
    

df_dila["Message d'origine"] = df_dila["Message d'origine"].apply(process_text)

In [344]:
BUCKET_OUT = "projet-transformers"
FILE_KEY_OUT_S3 = "test_dila_output.csv"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    df_dila.to_csv(file_out,
    #encoding="utf-8",
    #sep=";"
    )

In [341]:
df_dila

,Message d'origine
N°,
1,""" Bonjour, je viens d'arriver en XXXXXX ,j'aie..."
2,""" J'ai fait une demande de duplicata de carte ..."
3,""" Bonjour, J'ai passé commande pour certificat..."
4,""" Bonjour, Mr XXXXXXXXXXXXXXX est inscrit chez..."
5,""" En date duXXXXXXXXX, j'ai commandé deux vign..."
...,...
96,""" Bonjour, J'aimerais savoir si ce dimanche 26..."
97,""" J'ai acheté un véhicule leXXXXXXX:19. L'anci..."
98,""" Bonjour, J'ai fait une demande de copie d'in..."
